### 1 - Collecte des données

In [1]:
# Blocage de la sauvegarde automatique.
%autosave 0

# Importation des librairies nécessaires.
from bs4 import BeautifulSoup as bs
import requests 
import time
import pandas as pd
import datetime as dt

# Création de l'objet BeautifulSoup dans une variable nommée soup.
url = "https://fr.trustpilot.com/review/www.centralepneus.fr"
page = requests.get(url)  
soup = bs(page.content, "lxml")

# Récupération des informations clés de la page.
nom_entreprise = soup.find('span', attrs = {'class' : "typography_display-s__qOjh6 typography_appearance-default__AAY17 title_displayName__TtDDM"}).text
nombre_avis = soup.find('p', attrs = {'class' : "typography_body-l__KUYFJ typography_appearance-default__AAY17"}).text
moyenne_note = soup.find('span', attrs = {'class' : "typography_heading-m__T_L_X typography_appearance-default__AAY17"}).text
print("Nom de l'entreprise :", nom_entreprise)
print("Nombre d'avis :", nombre_avis)
print("Moyenne des notes :", moyenne_note)

Autosave disabled
Nom de l'entreprise : CentralePneus.fr 
Nombre d'avis : Total : 258 512
Moyenne des notes : 4,3


In [2]:
# Collecte des informations sur Allopneus:
url = "https://fr.trustpilot.com/review/www.centralepneus.fr"
from_page = 1
to_page = 9866
nom_clients,nombre_avis_publiés,notes,titres,commentaires,date_expériences,date_publications,localisations,réponses,date_réponses,pays,statut_vérifié = [],[],[],[],[],[],[],[],[],[],[],[]  

for page in range(from_page, to_page + 1):
    url_page = f'{url}?page={page}'
    response = requests.get(url_page)
    soup = bs(response.content, 'html.parser') 
    avis_client = soup.find_all('div', attrs = {'class': "styles_reviewCardInner__EwDq2"})

    for avis in avis_client: 
        nom_client_element = avis.find('span',class_='typography_heading-xxs__QKBS8 typography_appearance-default__AAY17')
        nom_client = nom_client_element.text.strip() if nom_client_element else None
        nom_clients.append(nom_client)
        nombre_avis_publié = avis.find('span',class_='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l').text.strip() 
        nombre_avis_publiés.append(nombre_avis_publié)
        note = avis.find(class_ = "star-rating_starRating__4rrcf star-rating_medium__iN6Ty").findChild() 
        notes.append(note["alt"])
        titre = avis.find('h2',class_='typography_heading-s__f7029 typography_appearance-default__AAY17').text.strip() 
        titres.append(titre)
        commentaire_element = avis.find('p',class_='typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn')
        commentaire = commentaire_element.text.strip() if commentaire_element else None   
        commentaires.append(commentaire)
        pays_element = avis.find(lambda tag: tag.name=='svg' and tag.get('class') == ['icon_icon__ECGRl'])
        verif_element = avis.find(lambda tag: tag.name=='svg' and tag.get('class') == ['icon_icon__ECGRl']).find_next('svg') if avis.find(lambda tag: tag.name=='svg' and tag.get('class') == ['icon_icon__ECGRl']) else None
        if pays_element:
            pays.append(pays_element.find_next('span').text.strip())
        else:
            pays.append(None)
        if verif_element:
            statut_vérifié.append(1) if verif_element.find_next('span').text.strip() == 'Vérifié' else statut_vérifié.append(0)
        else:
            statut_vérifié.append(0)     
        date_expérience_element = avis.find('p',class_='typography_body-m__xgxZ_ typography_appearance-default__AAY17') 
        date_expérience = date_expérience_element.text.strip() if date_expérience_element else None
        date_expériences.append(date_expérience)
        date_publication_element = avis.find('time', class_="")
        if date_publication_element:
            date_publication = date_publication_element.get('datetime')
            date_publications.append(date_publication)
        else:
            date_publications.append(None)  
        réponse_element = avis.find('p', class_='typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX')
        if réponse_element:
            réponse = réponse_element.text.strip()
            réponses.append(réponse)
        else:
            réponses.append(None)     
        date_réponse_element = avis.find('time', class_='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_replyDate__Iem0_')  
        if date_réponse_element:
            date_réponse = date_réponse_element.get('datetime')
            date_réponses.append(date_réponse)
        else:
            date_réponses.append(None)
        time.sleep(0.2)

# Intégration des données dans un DataFrame.
df_Centralepneus = pd.DataFrame(list(zip(nom_clients,nombre_avis_publiés,notes,titres,commentaires,date_expériences,date_publications,réponses,date_réponses,pays,statut_vérifié)),
               columns =["Client","Nombre d'avis publié","Note","Titres","Commentaire","Date de l'expérience","Date de publication","Réponse","Date de réponse","Pays","Vérifications"])

# Ajout du nom de l'entreprise évaluée dans nouvelle colonne.
df_Centralepneus["Entreprise"] = nom_entreprise

In [3]:
# Vérification : est-ce que l'ensemble des avis a bien été collecté ?
df_Centralepneus.info()

# Exportation du DataFrame dans un fichier Excel.
df_Centralepneus.to_csv(r'C:\Users\micka\OneDrive\Bureau\df_Centralepneus.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197140 entries, 0 to 197139
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Client                197140 non-null  object
 1   Nombre d'avis publié  197140 non-null  object
 2   Note                  197140 non-null  object
 3   Titres                197140 non-null  object
 4   Commentaire           159138 non-null  object
 5   Date de l'expérience  197140 non-null  object
 6   Date de publication   197140 non-null  object
 7   Réponse               33922 non-null   object
 8   Date de réponse       33922 non-null   object
 9   Pays                  197137 non-null  object
 10  Vérifications         197140 non-null  int64 
 11  Entreprise            197140 non-null  object
dtypes: int64(1), object(11)
memory usage: 18.0+ MB
